In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [2]:
from transformers import BertTokenizerFast, BertForTokenClassification, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import TextDataset

In [ ]:
!pip install python-docx

In [4]:
import json
# Read the JSON file
path = "/content/drive/MyDrive/model/B_data.json"
with open(path, 'r') as f:
    dataset = json.load(f)

In [8]:
text, intent, ner = [], [], []
for i in dataset:
    text.append(i['text'])
    intent.append(i['intent'])
    ner.append(i['entities'].split())

In [13]:
unique_intents = set(intent)
num_intent_labels = len(unique_intents)

num_intent_labels

5

In [15]:
one_dimensional_ner = [tag for subset in ner for tag in subset ]
unique_ner = set(one_dimensional_ner)
num_ner_labels = len(unique_ner)
unique_ner, num_ner_labels

({'B-DATE',
  'B-DUR',
  'B-TASK',
  'B-TIME',
  'I-DATE',
  'I-DUR',
  'I-TASK',
  'I-TIME',
  'O'},
 9)

In [16]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
ner_model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=num_ner_labels)
intent_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_intent_labels)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
label_ner_map = {'O': 0, 'B-DATE': 1, 'I-DATE': 2, 'B-TIME': 3, 'I-TIME': 4, 'B-TASK': 5, 'I-TASK': 6, 'B-DUR': 7, 'I-DUR': 8}

In [ ]:
class dataset(Dataset):
    def __init__(self, text, intent, ner, tokenizer, max_len):
        self.len = len(text)
        self.text = text
        self.intent = intent
        self.ner = ner
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence, ner label, and intent_label
        sentence = self.text[index].strip().split()
        intent_label = self.intent[index].strip()
        ner_labels = self.ner[index]

        # step 2: use tokenizer to encode a sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" which highlights where each token starts and ends
        encoding = self.tokenizer(
            sentence,
            is_pretokenized=True, # in case of an error I will set it to False
            return_offsets_mapping=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_len
        )

        # step 3: create ner token labels only for first word pieces of each tokenized word
        tokenized_ner_labels = [label_ner_map[label] for label in ner_labels]
